In [20]:
import numpy as np
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
from dateutil import tz
from collections import defaultdict

# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('cms_key.json', scope)

In [11]:
sheet_name = "Content_datathon2023_scoreboard"
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
wb = client.open(sheet_name)
datasheet = wb.worksheet("submissions")
list_of_lists = datasheet.get_all_values()

In [16]:
team = "Testers 2"
score = 55.2
այժմ = datetime.now().astimezone(tz.gettz("Asia/Yerevan")).strftime("%Y-%m-%d %H:%M:%S")

if len(list_of_lists) == 1:
    list_of_lists.append([team, score, այժմ])
    datasheet.update(list_of_lists)
else:
    dataframe = pd.DataFrame(datasheet.get_all_records())
    dataframe = dataframe.append({"Team": team, "Score": score, "Time": այժմ}, ignore_index=True)
    
    team_max = dataframe.groupby("Team").max("Score").reset_index()
    
    dataframe = dataframe[dataframe[["Team", "Score"]].isin(team_max).all(axis=1)]
    dataframe = dataframe.sort_values("Score", ascending=False)
    
    # datasheet.update([dataframe.columns.values.tolist()] + dataframe.values.tolist())
    
dataframe

        Team  Score                          Time
0  Testers 2   55.2  2023-02-05 17:11:39 GMT+0400


,Team,Score,Time
0,Testers 2,55.2,2023-02-05 17:11:39 GMT+0400


In [15]:
dataframe[["Team", "Score"]].isin(team_max).all(axis=1)

0     True
1    False
dtype: bool

In [ ]:
teamsheet = wb.worksheet("teams")
teams = set([i[0] for i in teamsheet.get_values("A2:A1000")])
teams

{'Admins', 'Moderators'}

In [14]:
dataframe[dataframe.Text == "Dessert is coming"]

,Text,Label
1395,Dessert is coming,0


In [ ]:
dataframe.loc[scores_df.Team == team]

In [18]:
AM_TZ = tz.gettz("Asia/Yerevan")
(այժմ := datetime.now().astimezone(AM_TZ))

datetime.datetime(2023, 2, 5, 17, 48, 29, 387560, tzinfo=tzfile('Asia/Yerevan'))

In [ ]:
# region Inits

# filenames
SCOREBOARD_DS_NAME = "Content_datathon2023_scoreboard"
EVAL_DS_NAME = "Eval_datathon2023"
EVAL_DF_PATH = "data/eval_datathon2023.pkl"
PRIVATE_PATHS = "private_paths" # deleted later
SCOREBOARD_PATH = "data/scoreboard2023.pkl"
TEAMBOARD_PATH = "data/teamboard2023.pkl"
CMS_KEY = "cms_key.json"


# use creds to create a client to interact with the Google Drive API
SCOPE = ['https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive']
CREDS = ServiceAccountCredentials.from_json_keyfile_name(CMS_KEY, scope)

AM_TZ = tz.gettz("Asia/Yerevan")

team_passwords = defaultdict(lambda: [])

# setting non-public path for updating the dataset to 
# avoid google sheet api limit hits by other parties 
with open(PRIVATE_PATHS, "r") as f:
    p_paths = f.readlines()
del PRIVATE_PATHS

#endregion Inits

client = gspread.authorize(CREDS)
wb = client.open(EVAL_DS_NAME)
datasheet = wb.get_worksheet(0)
dataframe = pd.DataFrame(datasheet.get_all_records())

dataframe.to_pickle(EVAL_DF_PATH)
dataframe;

In [33]:
import json
json.dumps({"data": dataframe.loc[250:350].X.to_list()});

In [55]:
AM_TZ = tz.gettz("Asia/Yerevan")
TASK_REVEAL = datetime(2023, 2, 10, 19, 0, tzinfo = AM_TZ)
ACCEPTING_SUBMISSIONS = datetime(2023, 2, 11, 17, 40, tzinfo = AM_TZ)
END_OF_SUBMISSIONS = datetime(2023, 2, 11, 18, 15, tzinfo = AM_TZ)
eval_df = dataframe.copy()

def serve_data():
    """This is the endpoint used by participants to fetch evaluation data. 
    (No labels will be sent out)"""
    
    այժմ = datetime.now().astimezone(AM_TZ)
    
    
    if այժմ < TASK_REVEAL:
        return json.dumps({"message": "I value your perspicacity and interest, " + \
            "but the task is not yet revealed."})
    else:
        if այժմ > END_OF_SUBMISSIONS:
            return json.dumps({"message": "The datathon has ended :( \n" + \
                "Contact Mher Movsisyan if you think this is a mistake."})
        
        if այժմ < ACCEPTING_SUBMISSIONS:
            return json.dumps({
                "data": eval_df.loc[250:350].X.to_list(), 
                "message": "This is not all the data, once the submission process" + \
                    " begins you will be given a larger dataset through this endpoint."})
            
        return json.dumps({
            "data": eval_df.X.to_list(), 
            "message": "Good luck!"})
        
print(json.loads(serve_data())["message"])
len(json.loads(serve_data())["data"])

I value your perspicacity and interest, but the task is not yet revealed.


KeyError: 'data'

In [44]:
datetime(2023, 2, 10, 19, 1, tzinfo = AM_TZ)

datetime.datetime(2023, 2, 10, 19, 1, tzinfo=tzfile('Asia/Yerevan'))

In [61]:
client = gspread.authorize(CREDS)
wb = client.open(SCOREBOARD_DS_NAME)

# get teams and their participants
teamsheet = wb.worksheet("teams")
teams_df = pd.DataFrame(teamsheet.get_all_records())
    
# teams_df.to_pickle(TEAMBOARD_PATH)
teams_df;

In [58]:
for i, team, name, password in teams_df.itertuples():
    team_passwords[team] += [password]

In [60]:
team_passwords;

In [69]:
submissions = wb.worksheet("submissions")
score_df = pd.DataFrame(submissions.get_all_records())
score_df.Score = score_df.Score.astype("float")

score_df

,Team,Score,Time
0,Testers 2,55.2000,2023-02-05 17:11:39
1,Admins,0.7432,2023-02-05 21:09:28
2,Admins,0.7432,2023-02-05 21:11:21
3,Admins,0.7432,2023-02-05 21:12:08
4,Admins,0.7432,2023-02-05 21:12:51
5,Admins,0.7432,2023-02-05 21:13:53
6,Admins,0.7432,2023-02-05 21:14:57
7,Admins,0.7432,2023-02-05 21:16:47
8,Admins,74.3200,2023-02-05 21:18:55


In [66]:
# Get the maximum per team
team_max = score_df.groupby("Team").max("Score").reset_index()
team_max

,Team,Score
0,Admins,74.32
1,Testers 2,55.20


In [67]:
score_df = score_df[score_df[["Team", "Score"]].isin(team_max).all(axis=1)]

In [73]:
score_df[["Team", "Score"]].isin(team_max)

,Team,Score
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False


In [75]:
score_df[["Team", "Score"]].stack().isin(team_max.stack().values).unstack()

0  Team     Testers 2
   Score         55.2
1  Team        Admins
   Score       0.7432
2  Team        Admins
   Score       0.7432
3  Team        Admins
   Score       0.7432
4  Team        Admins
   Score       0.7432
5  Team        Admins
   Score       0.7432
6  Team        Admins
   Score       0.7432
7  Team        Admins
   Score       0.7432
8  Team        Admins
   Score        74.32
dtype: object

In [ ]:
# Sort
score_df = score_df.sort_values("Score", ascending=False)